In [6]:
%pwd

'/Users/nguyenphungbaohuy/Desktop/python/Medical-Chatbot/research'

In [7]:
import os
os.chdir("../")

In [8]:
%pwd

'/Users/nguyenphungbaohuy/Desktop/python/Medical-Chatbot'

In [9]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
# Extract Data From the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents=loader.load()
    
    return documents

In [11]:
extracted_data = load_pdf_file(data='Data/')

In [12]:
# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks 

In [13]:
text_chunks = text_split(extracted_data=extracted_data)
print("Length of Text Chunk", len(text_chunks))

Length of Text Chunk 5860


In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
# Downliad the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [16]:
embeddings = download_hugging_face_embeddings()

/var/folders/w9/d3h7vx393175sc9kvpd02znh0000gn/T/ipykernel_10220/3818717040.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Users/nguyenphungbaohuy/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
PINECONE_API_KEY = os.environ.get('PINECONE_API')
HUGGING_FACE_API = os.environ.get('HUGGING_FACE_API')

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

In [ ]:
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [18]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [21]:
# Load Existing index
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [22]:
docsearch

In [23]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [24]:
retrieved_docs = retriever.invoke("What is Acne?")

In [25]:
retrieved_docs

[Document(id='6f83f40b-ea6a-4a7e-bcab-b68d8805a35b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='67815ce8-a344-41bb-9987-005b5aa33222', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='20e32dc6-952b-4d0c-81c3-c671711d1593', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'p

In [2]:
from langchain_community.llms import Ollama
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [3]:
# Initialize Ollama LLM (Change "mistral" to other models if needed)
llm = Ollama(model="llama3.2", temperature=0.4)

/var/folders/w9/d3h7vx393175sc9kvpd02znh0000gn/T/ipykernel_10220/726980539.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2", temperature=0.4)


In [26]:
# Define system prompt for retrieval-based answering
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following retrieved context to answer the question. "
    "If you don't know, say you don't know. Use at most three sentences. "
    "Keep the answer concise.\n\n"
    "{context}"
)

# Create prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


In [27]:
# Create document-based QA chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
# Assume `retriever` is defined elsewhere and retrieves relevant documents
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [28]:
# Query the RAG system
response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
# Print the generated answer
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a particular chemical from the pituitary gland in the brain, leading to increased growth in bone and soft tissue, as well as various other bodily disturbances. It affects humans, causing symptoms such as excessive height, joint pain, and organ damage. The condition can have significant health consequences if left untreated.


In [29]:
reponse_1 = rag_chain.invoke({"input": "What is stats?"})
print(reponse_1["answer"])

I don't know what "stats" refers to in this context. The retrieved text only discusses a complete blood count (CBC) as part of a routine physical examination. It does not mention any other meaning of "stats".
